In [4]:
#imports
from pyspark.sql import SparkSession

In [5]:
#instanciando spark 
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('spark IGTI') \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

23/06/03 00:52:36 WARN Utils: Your hostname, caio-HP-250-G8-Notebook-PC resolves to a loopback address: 127.0.1.1; using 192.168.1.122 instead (on interface wlo1)
23/06/03 00:52:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/03 00:52:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/03 00:52:38 WARN Utils: Service 'SparkUI' could not bind on port 4050. Attempting port 4051.


In [6]:
file_dir = 'base_dados/README.md'

In [8]:
linesRdd = spark.sparkContext.textFile(file_dir)

#contando linhas
linesRdd.count()

125

In [10]:
#map
#esta operação quebra as linhas do texto e coloca em uma tupla juntamento com o tamanho da linha em questão
mapRdd = linesRdd.map(lambda line: (line, len(line)))
mapRdd.collect()


[('# Apache Spark', 14),
 ('', 0),
 ('Spark is a unified analytics engine for large-scale data processing. It provides',
  80),
 ('high-level APIs in Scala, Java, Python, and R, and an optimized engine that',
  75),
 ('supports general computation graphs for data analysis. It also supports a',
  73),
 ('rich set of higher-level tools including Spark SQL for SQL and DataFrames,',
  74),
 ('pandas API on Spark for pandas workloads, MLlib for machine learning, GraphX for graph processing,',
  98),
 ('and Structured Streaming for stream processing.', 47),
 ('', 0),
 ('<https://spark.apache.org/>', 27),
 ('', 0),
 ('[![GitHub Actions Build](https://github.com/apache/spark/actions/workflows/build_main.yml/badge.svg)](https://github.com/apache/spark/actions/workflows/build_main.yml)',
  167),
 ('[![AppVeyor Build](https://img.shields.io/appveyor/ci/ApacheSoftwareFoundation/spark/master.svg?style=plastic&logo=appveyor)](https://ci.appveyor.com/project/ApacheSoftwareFoundation/spark)',
  189),


In [11]:
"""
flatmap 

quebrando as linhas em palavras
"""


FlatmapRdd = linesRdd.flatMap(lambda line: (line.split()))
FlatmapRdd.collect()

['#',
 'Apache',
 'Spark',
 'Spark',
 'is',
 'a',
 'unified',
 'analytics',
 'engine',
 'for',
 'large-scale',
 'data',
 'processing.',
 'It',
 'provides',
 'high-level',
 'APIs',
 'in',
 'Scala,',
 'Java,',
 'Python,',
 'and',
 'R,',
 'and',
 'an',
 'optimized',
 'engine',
 'that',
 'supports',
 'general',
 'computation',
 'graphs',
 'for',
 'data',
 'analysis.',
 'It',
 'also',
 'supports',
 'a',
 'rich',
 'set',
 'of',
 'higher-level',
 'tools',
 'including',
 'Spark',
 'SQL',
 'for',
 'SQL',
 'and',
 'DataFrames,',
 'pandas',
 'API',
 'on',
 'Spark',
 'for',
 'pandas',
 'workloads,',
 'MLlib',
 'for',
 'machine',
 'learning,',
 'GraphX',
 'for',
 'graph',
 'processing,',
 'and',
 'Structured',
 'Streaming',
 'for',
 'stream',
 'processing.',
 '<https://spark.apache.org/>',
 '[![GitHub',
 'Actions',
 'Build](https://github.com/apache/spark/actions/workflows/build_main.yml/badge.svg)](https://github.com/apache/spark/actions/workflows/build_main.yml)',
 '[![AppVeyor',
 'Build](https:/

In [12]:
"""
Filtrando apenas palavras que comece com a letra A
"""

filterRdd = (
    linesRdd
    .flatMap(lambda line: line.split())
    .filter(lambda word: word.startswith('a'))
)

filterRdd.collect()

['a',
 'analytics',
 'and',
 'and',
 'an',
 'analysis.',
 'also',
 'a',
 'and',
 'and',
 'a',
 'and',
 'a',
 'available',
 'at',
 'an',
 'also',
 'also',
 'a',
 'a',
 'and',
 'also',
 'an',
 'abbreviated',
 'are',
 'a',
 'also',
 'a',
 'and',
 'against',
 'at',
 'and',
 'a',
 'and',
 'an']

In [13]:
"""
Contando quantas vezes as palavras aparecem dentro do dataset
"""
count_words = ['um','dois','tres','um','tres']

rdd = spark.sparkContext.parallelize(count_words)

rdd2 = (
    rdd
    .map(lambda word: (word,1))
    .reduceByKey(lambda a,b: a+b)
)

rdd2.collect()



[('um', 2), ('dois', 1), ('tres', 2)]

In [14]:
"""
Contando quantas vezes as palavras aparecem dentro do dataset e ordenando pela chave
"""
count_words = ['um','dois','tres','um','tres']

rdd = spark.sparkContext.parallelize(count_words)

rdd2 = (
    rdd
    .map(lambda word: (word,1))
    .reduceByKey(lambda a,b: a+b)
    .sortByKey('asc')
)

rdd2.collect()



[('dois', 1), ('tres', 2), ('um', 2)]

In [15]:
"""
Contando quantas vezes as palavras aparecem dentro do dataset e ordenando pelo segundo valor da chave
"""
count_words = ['um','dois','tres','um','tres']

rdd = spark.sparkContext.parallelize(count_words)

rdd2 = (
    rdd
    .map(lambda word: (word,1))
    .reduceByKey(lambda a,b: a+b)
    .sortBy(lambda t: t[1])
)

rdd2.collect()



[('dois', 1), ('um', 2), ('tres', 2)]

In [16]:
"""
Fazendo a união de dois datasets RDD em um unico
"""


df_1 = ['um','um','dois','tres']
df_2 = ['quatro','cinco']

rdd1 = spark.sparkContext.parallelize(df_1)
rdd2 = spark.sparkContext.parallelize(df_2)

rddUnion = rdd1.union(rdd2)

rddUnion.collect()


['um', 'um', 'dois', 'tres', 'quatro', 'cinco']

In [20]:
"""
localizando o item que interocorre entre os dois dadaset (localiza quais valores aparecem em ambos os datasets)
"""

df_1 = ['um','um','dois','tres','quatro']


rdd1 = spark.sparkContext.parallelize(df_1)
rdd2 = spark.sparkContext.parallelize(df_2)

rddUnion = rdd1.intersection(rdd2)

rddUnion.collect()



['um', 'quatro']

In [22]:
"""
localizando os item distintos no dataset
"""

df_1 = ['um','um','dois','tres','quatro']


rdd1 = spark.sparkContext.parallelize(df_1)


rddDistinct = rdd1.distinct()

rddDistinct.collect()


['um', 'dois', 'tres', 'quatro']

In [24]:
"""
Operação de junção entre datasets (JOIN)
"""

lista1 = [('caio',21),('Istefania',19)]
lista2 = [('caio','SP'),('Istefania','SP'),('Joao','CE')]

rdd1 = spark.sparkContext.parallelize(lista1)
rdd2 = spark.sparkContext.parallelize(lista2)

rddJoin = rdd1.join(rdd2)

rddJoin.collect()

[('Istefania', (19, 'SP')), ('caio', (21, 'SP'))]

In [25]:
#contando valores

rddJoin.count()

2

In [26]:
#resgatando valores aleatorios do rdd TAKE

rddJoin.take(1)



[('Istefania', (19, 'SP'))]

In [27]:
"""
Resgantando os 3 primeiro valores (TOP)
"""

df_1 = ['um','um','dois','tres']
df_2 = ['quatro','cinco']

rdd1 = spark.sparkContext.parallelize(df_1)
rdd2 = spark.sparkContext.parallelize(df_2)

rddUnion = rdd1.union(rdd2)

rddUnion.collect()

rddUnion.top(3)


['um', 'um', 'tres']

In [28]:
"""
Contando quantas vezes cada registro ocorreu
"""

df_1 = ['um','um','dois','tres']
df_2 = ['quatro','cinco']

rdd1 = spark.sparkContext.parallelize(df_1)
rdd2 = spark.sparkContext.parallelize(df_2)

rddUnion = rdd1.union(rdd2)

rddUnion.collect()

rddUnion.countByValue()


defaultdict(int, {'um': 2, 'dois': 1, 'tres': 1, 'quatro': 1, 'cinco': 1})

In [30]:
"""
Gravando as transformações do rdd como um arquivo
"""
rddUnion.saveAsTextFile('base_dados/saida2.txt')
